In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['LANCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")# thsi is fr langsmith trackig
os.environ['LANGCHAIN_TRACING_V2'] = "true" # required for langchain 
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT") # required for langsmith tracking and we have created tis name in dotenv  

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x00000215BE5B3760> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000215C24A6CB0> root_client=<openai.OpenAI object at 0x00000215BE5B2920> root_async_client=<openai.AsyncOpenAI object at 0x00000215C24A6C20> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [3]:
# data ingestion from website https://docs.smith.langchain.com/administration/tutorials/manage_spend
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")

loader 


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended R

load data---->docs--->divide in chunks---->text---->vector----->vector embedding-----vector store DB

In [5]:
# we need to divide the documents into smaller chunks, so that we can use them in retriever because retriever works on smaller chunks of text

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
docs = text_splitter.split_documents(docs)
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended ReadingBefo

In [6]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings() # text to vectors

In [7]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(docs, embeddings) # vector store db 

In [8]:
vector_store

In [9]:
query = "LangSmith has two usage limits: total traces and extended"
result = vector_store.similarity_search(query)
result[0].page_content

'Optimization 2: limit usage\u200b\nIn the previous section, we managed data retention settings to optimize existing spend. In this section, we will\nuse usage limits to prevent future overspend.\nLangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we\'ve\nbeen tracking on our usage graph. We can use these in tandem to have granular control over spend.\nTo set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the\nbottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear, along\nwith a cost estimate:\n\nLets start by setting limits on our production usage, since that is where the majority of spend comes from.\nSetting a good total traces limit\u200b\nPicking the right "total traces" limit depends on the expected load of traces that you will send to LangSmith. You should\nclearly think about your assumptions before setting a lim

User Input → Prompt Template → LLM → Output


Key Components of a Chain
Prompt Template
Structures the input for the model (e.g., using ChatPromptTemplate).

LLM or ChatModel
The actual language model used (e.g., OpenAI’s GPT, Anthropic’s Claude, etc.).

Output Parser (optional)
Parses the LLM's output into structured data (e.g., JSON, Pydantic models).

Memory (optional)
Stores chat history or contextual information across interactions.

Tools (optional)
Enables agents or chains to interact with external tools or APIs.

# retrieval chain (below Chain style is for RAG)

In [10]:
# using retrieval chain, document chaining, and llm
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# from_template is used to create one line prompt template
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based on the context below.
    <context>
    {context}
    </context>    
    """
)


doc_chain = create_stuff_documents_chain(llm, prompt)
doc_chain

## thi doc_chain will take the query and context and return the answer

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the context below.\n    <context>\n    {context}\n    </context>    \n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000215BE5B3760>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000215C24A6CB0>, root_client=<openai.OpenAI object at 0x00000215BE5B2920>, root_async_client=<openai.AsyncOpenAI object at 0x00000215C24A6C20>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwar

In [11]:
# this piece is for demo purpose , not used further
# here you can see that we are passing the context manually, but in real world application, we will use retriever to get the context dynamically based on the query 
from langchain_core.documents import Document


doc_chain.invoke({
    "input":"what is LangSmith has two usage limits: total traces and extended",
    "context": [Document(page_content="LangSmith has two usage limits: total traces and extended traces. The total traces limit is the maximum number of traces that can be stored in LangSmith, while the extended traces limit is the maximum number of traces that can be stored in LangSmith for a specific project. The total traces limit is 100,000 traces, while the extended traces limit is 10,000 traces per project.")]
})
# context added manually 



'What happens if a user exceeds the extended traces limit for a specific project in LangSmith?'

however we want the docuents to first come from the retriever we just set up. That way , we can use the retriever to dynamically select the most relevant documents and pass those in for a given question

In [12]:
## input--->retriever--->vectorDB 

Essentially, retrieval_chain orchestrates the entire process:

input --> retriever (gets context) --> doc_chain (uses context and original input to generate answer)



In [13]:
retriever = vector_store.as_retriever()# so we are making the vector store as context 
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, doc_chain)

In [14]:
retrieval_chain # it is just a chain that takes the input and returns the answer based on the context from the retriever

# main part is retriever it retreves relevant content from vector store based on the query and then passes it to the doc_chain which returns the answer based on the context


RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000215D1A4B7F0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the context below.\n    <context>\n    {context}\n    </context>    \n    '), additional_kwargs={})])
  

In [15]:
# get the answer from the retriever
response = retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

"Based on the context provided, if you have any specific questions on setting usage limits, understanding your current usage with LangSmith, or managing spend using LangSmith's usage metrics, feel free to ask! The context outlines the process of setting up usage limits, understanding spend via usage graphs and invoices, and the importance of managing these metrics for environments like production that incur the majority of the costs."

In [16]:
response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(id='ac4606b6-fbc4-413f-b99c-d40cda6176e3', metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimization 2: limit usage\u200b\nIn the previous section, we managed data retention settings to optimize existing spend. In this section, we will\nuse usage limits to prevent future overspend.\nLangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we\'ve\nbeen tracking on our usage graph. We can use these in tandem to have granular control over spend.\nTo set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the\nbottom of the page that lets you set usage limits

# Understanding

vector_store = docs is embedded

prompt = from_template("""...{context}""")

docs_chain = create_stuff_document_chain(prompt + llm)

retriever = vectorDB #this search things from vectorstore that are relevant t our qestion and pass it as our context in prompt

retrieval_chain = retriever + docs_chain (just a chain)

This is where the magic happens. It creates a complete workflow:

Step 1 (Retrieval): When retrieval_chain.invoke({"input": "your question"}) is called, the retrieval_chain first uses the retriever to search for documents based on "your question". The retriever will find the document that talks about "LangSmith usage limits" because it matches your input.

Step 2 (Context Passing): The documents found by the retriever are then automatically passed as the context argument to the doc_chain. You don't see this explicit passing in your retrieval_chain.invoke() call because create_retrieval_chain handles it internally.

Step 3 (Document Stuffing/Answering): The doc_chain then takes these retrieved documents (as its context) and your original "input" (your question) and uses the llm to formulate an answer.

This "input" (your question) is used in two main ways within the retrieval_chain:

By the retriever: The retriever uses this question to search your vector_store and find the most relevant documents (your context).

By the doc_chain: Once the relevant documents are found, the doc_chain receives both these documents (as context) and your original question (as input) to formulate the final answer using the language model.